1. Vol
2. AVol
3. RVol

In [4]:
import os
import polars as pl
from tqdm import tqdm
from datetime import datetime

In [6]:
for code in tqdm(os.listdir(f'cn_data/')):
    dates = []
    vols = []

    for fname in os.listdir(f'cn_data/{code}'):
        date_str = fname.split('.')[0]

        df = pl.read_csv(f'cn_data/{code}/{fname}')

        rs = df.filter((pl.col('time_str') == '09:25') | (pl.col('time_str') == '09:30') | (pl.col('time_str') == '09:31') | (pl.col('time_str') == '09:32') | (pl.col('time_str') == '09:33') | (pl.col('time_str') == '09:34') | (pl.col('time_str') == '09:35'))

        vol = rs['vol'].sum()
        
        dates.append(date_str)
        vols.append(vol)

    df = pl.DataFrame({
        'date_str': dates,
        'vol': vols
    }).sort('date_str')

    avol_30 = df['vol'].shift(1).rolling_mean(30).round(3)
    df['avol_30'] = avol_30.to_list()
    df['rvol_30'] = (df['vol'] / df['avol_30']).round(3)

    #
    df['avol_30'] = df['avol_30'].fill_nan(0.0).fill_null(0.0)
    df['rvol_30'] = df['rvol_30'].fill_nan(0.0).fill_null(0.0)

    #
    df['vol'] = df['vol'].apply(lambda x: int(x))

    #
    df['freq'] = ['1d'] * len(df)

    #
    dts = []
    for tstr in df['date_str']:
        dts.append(datetime.strptime(tstr, '%Y%m%d'))
    df['time'] = dts
    
    df.to_csv(f'cn_data_rvol/data/{code}.csv')
    df.to_parquet(f'cn_data_rvol/data_parquet/{code}.parquet')

100%|██████████| 1925/1925 [03:49<00:00,  8.37it/s]
